In [1]:
import struct
import copy
import json
import os
import shutil
import time

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# CUSTOM
from utils import *
from baxter_pickplace_interface import *
from pydpp.dpp import DPP

# ROS
import rospy
import gazebo_msgs.srv
import geometry_msgs.msg
import std_msgs.msg
import tf.transformations

# BAXTER
import baxter_core_msgs.srv
import baxter_interface

# GQCNN
from autolab_core import YamlConfig, Logger
import autolab_core.utils as utils
from gqcnn import get_gqcnn_model, get_gqcnn_trainer, utils as gqcnn_utils
from perception import (BinaryImage, CameraIntrinsics, ColorImage, DepthImage,
                        RgbdImage)
from visualization import Visualizer2D as vis
from gqcnn.grasping import GraspAction
from gqcnn.grasping import (RobustGraspingPolicy,
                            CrossEntropyRobustGraspingPolicy, RgbdImageState,
                            FullyConvolutionalGraspingPolicyParallelJaw,
                            FullyConvolutionalGraspingPolicySuction)
from gqcnn.utils import GripperMode
from gqcnn import NoValidGraspsException

# FOR VISUALIZATION
from matplotlib import pyplot as plt
%matplotlib inline

/home/dof6/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
W0905 18:47:13.961113 140170639537920 deprecation_wrapper.py:119] From /home/dof6/.local/lib/python2.7/site-packages/gqcnn/utils/enums.py:44: The name tf.RunOptions is deprecated. Please use tf.compat.v1.RunOptions instead.

I0905 18:47:13.972800 140170639537920 driver.py:120] Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt


root       INFO     Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt


I0905 18:47:13.990762 140170639537920 driver.py:120] Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


root       INFO     Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


W0905 18:47:14.440557 140170639537920 kinect2_sensor.py:15] Unable to import pylibfreenect2. Python-only Kinect driver may not work properly.


...

root       WARNING  Failed to import ROS in phoxi_sensor.py. PhoXiSensor functionality unavailable.


I0905 18:47:14.494374 140170639537920 acceleratesupport.py:13] OpenGL_accelerate module loaded


OpenGL.acceleratesupport INFO     OpenGL_accelerate module loaded


I0905 18:47:14.498920 140170639537920 arraydatatype.py:270] Using accelerated ArrayDatatype


OpenGL.arrays.arraydatatype INFO     Using accelerated ArrayDatatype


In [2]:
# Image Cropping Info
cx = 285
cy = 255
wx = 115
wy = 135
crop_size = int(96/2)
target_dir = './dynamic_grasp_data/'
fname = ['tf_depth_ims_', 'grasps_', 'grasp_metrics_']
hand_pose_dim = 6

def load_policy(iteration, wx = 115, wy = 135, method = ''):
    if iteration==0:
        model_name = 'GQCNN-4.0-PJ'
    else:
        model_name = 'GQCNN-4.0-PJ-'+str(iteration)+'-'+method+"-GQ"
    config_filename = None
    fully_conv = False

    __file__ = os.path.abspath('')

    depth_im_filename = None
    segmask_filename = None
    camera_intr_filename = None
    if iteration == 0:
        model_dir = None
    else:
        model_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)),"scripts/grasp_learner/models/")

    assert not (fully_conv and depth_im_filename is not None
                and segmask_filename is None
                ), "Fully-Convolutional policy expects a segmask."

    if camera_intr_filename is None:
        camera_intr_filename = os.path.join(os.path.dirname(os.path.realpath(__file__)),
                                            "scripts/grasp_experts/data/calib/primesense/primesense.intr")

    # Set model if provided.
    if model_dir is None:
        model_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)),
                                 "scripts/grasp_experts/models")
    model_path = os.path.join(model_dir, model_name)

    # Get configs.
    model_config = json.load(open(os.path.join(model_path, "config.json"), "r"))
    try:
        gqcnn_config = model_config["gqcnn"]
        gripper_mode = gqcnn_config["gripper_mode"]
    except KeyError:
        gqcnn_config = model_config["gqcnn_config"]
        input_data_mode = gqcnn_config["input_data_mode"]
        if input_data_mode == "tf_image":
            gripper_mode = GripperMode.LEGACY_PARALLEL_JAW
        elif input_data_mode == "tf_image_suction":
            gripper_mode = GripperMode.LEGACY_SUCTION
        elif input_data_mode == "suction":
            gripper_mode = GripperMode.SUCTION
        elif input_data_mode == "multi_suction":
            gripper_mode = GripperMode.MULTI_SUCTION
        elif input_data_mode == "parallel_jaw":
            gripper_mode = GripperMode.PARALLEL_JAW
        else:
            raise ValueError(
                "Input data mode {} not supported!".format(input_data_mode))

    # Set config.
    if config_filename is None:
        if (gripper_mode == GripperMode.LEGACY_PARALLEL_JAW
                or gripper_mode == GripperMode.PARALLEL_JAW):
            if fully_conv:
                config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), "scripts/grasp_experts/cfg/examples/fc_gqcnn_pj.yaml")
            else:
                config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), "scripts/grasp_experts/cfg/examples/gqcnn_pj.yaml")
        elif (gripper_mode == GripperMode.LEGACY_SUCTION
              or gripper_mode == GripperMode.SUCTION):
            if fully_conv:
                config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), "scripts/grasp_experts/cfg/examples/fc_gqcnn_suction.yaml")
            else:
                config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), "scripts/grasp_experts/cfg/examples/gqcnn_suction.yaml")
    else:
        config_filename = os.path.join(
                    os.path.dirname(os.path.realpath(__file__)), config_filename)
    # Read config.
    config = YamlConfig(config_filename)
    inpaint_rescale_factor = config["inpaint_rescale_factor"]
    policy_config = config["policy"]

    # Make relative paths absolute.
    if "gqcnn_model" in policy_config["metric"]:
        policy_config["metric"]["gqcnn_model"] = model_path
        if not os.path.isabs(policy_config["metric"]["gqcnn_model"]):
            policy_config["metric"]["gqcnn_model"] = os.path.join(
                os.path.dirname(os.path.realpath(__file__)), "..",
                policy_config["metric"]["gqcnn_model"])

    # Setup sensor.
    camera_intr = CameraIntrinsics.load(camera_intr_filename)

    # Set input sizes for fully-convolutional policy.
    if fully_conv:
        policy_config["metric"]["fully_conv_gqcnn_config"][
            "im_height"] = wy*2
        policy_config["metric"]["fully_conv_gqcnn_config"][
            "im_width"] = wx*2

    # Init policy.
    if fully_conv:
        # TODO(vsatish): We should really be doing this in some factory policy.
        if policy_config["type"] == "fully_conv_suction":
            policy = FullyConvolutionalGraspingPolicySuction(policy_config)
        elif policy_config["type"] == "fully_conv_pj":
            policy = FullyConvolutionalGraspingPolicyParallelJaw(policy_config)
        else:
            raise ValueError(
                "Invalid fully-convolutional policy type: {}".format(
                    policy_config["type"]))
    else:
        policy_type = "cem"
        if "type" in policy_config:
            policy_type = policy_config["type"]
        if policy_type == "ranking":
            policy = RobustGraspingPolicy(policy_config)
        elif policy_type == "cem":
            policy = CrossEntropyRobustGraspingPolicy(policy_config)
        else:
            raise ValueError("Invalid policy type: {}".format(policy_type))
    print(policy_config)
            
    number_of_samples = 20
    def best_action(policy, depth_data, color_data):
        # Read images.
        depth_im = DepthImage(depth_data, frame=camera_intr.frame)
        color_im = ColorImage(color_data, frame=camera_intr.frame)

        # Inpaint.
        depth_im = depth_im.inpaint(rescale_factor=inpaint_rescale_factor)
        segmask = depth_im.invalid_pixel_mask().inverse()
        
        # Create state.
        rgbd_im = RgbdImage.from_color_and_depth(color_im, depth_im)
        state = RgbdImageState(rgbd_im, camera_intr, segmask=segmask)
        
        # Query policy.
        policy_start = time.time()
        grasps, q_values = policy.action_set(state)
            
        idx = np.argmax(q_values)
        grasp = grasps[idx]
        q_value = q_values[idx]
        
        action = GraspAction(grasp,q_value)
        print("Grasp Quality : %f"%q_value)
        
#         vis.figure(size=(10, 10))
#         depth_im = DepthImage(depth_data, frame=camera_intr.frame)
#         vis.imshow(depth_im,
#                    vmin=policy_config["vis"]["vmin"],
#                    vmax=policy_config["vis"]["vmax"])
#         vis.grasp(action.grasp, scale=2.5, show_center=True, show_axis=True)
#         vis.title("Planned grasp at depth {0:.3f}m with Q={1:.3f}".format(
#             action.grasp.depth, action.q_value))
#         vis.show()
        return action
    
    def sample_action(policy, depth_data, color_data):
        # Read images.
        depth_im = DepthImage(depth_data, frame=camera_intr.frame)
        color_im = ColorImage(color_data, frame=camera_intr.frame)

        # Inpaint.
        depth_im = depth_im.inpaint(rescale_factor=inpaint_rescale_factor)
        segmask = depth_im.invalid_pixel_mask().inverse()
        
        # Create state.
        rgbd_im = RgbdImage.from_color_and_depth(color_im, depth_im)
        state = RgbdImageState(rgbd_im, camera_intr, segmask=segmask)
        
        # Query policy.
        policy_start = time.time()
        grasps, q_values = policy.action_set(state)
            
        if method=='Greedy':
            idx = np.argmax(q_values)
            grasp = grasps[idx]
            q_value = q_values[idx]
        elif method=='Random':
            idx = np.random.choice(len(q_values),size=1)[0]
            grasp = grasps[idx]
            q_value = q_values[idx]
        elif method=='EXP3':
            p = np.exp((q_values - np.max(q_values))/0.005)
            p = p/np.sum(p)
            
            idx = np.random.choice(len(p),size=1,p=p)[0]
            grasp = grasps[idx]
            q_value = q_values[idx]

        action = GraspAction(grasp,q_value)
        print("Grasp Quality : %f"%q_value)
#         vis.figure(size=(10, 10))
#         depth_im = DepthImage(depth_data, frame=camera_intr.frame)
#         vis.imshow(depth_im,
#                    vmin=policy_config["vis"]["vmin"],
#                    vmax=policy_config["vis"]["vmax"])
#         vis.grasp(action.grasp, scale=2.5, show_center=True, show_axis=True)
#         vis.title("Planned grasp at depth {0:.3f}m with Q={1:.3f}".format(
#             action.grasp.depth, action.q_value))
#         vis.show()

        return action
            
    return policy, best_action, sample_action

def load_gqcnn_model(iteration,method=""):
    dataset_dir = "dynamic_grasp_data/"+method+"/"
    split_name = "image_wise"
    output_dir = "grasp_learner/models/"
    if iteration == 0:
        model_dir = "grasp_experts/models/"
    else:
        model_dir = "grasp_learner/models/"
    tensorboard_port = None
    seed = 1
    config_filename = "grasp_experts/cfg/finetune_dex-net_4.0_pj.yaml"
    name = "GQCNN-4.0-PJ-"+str(iteration+1)+'-'+method

    if iteration == 0:
        base_model_name = "GQCNN-4.0-PJ"
    else:
        base_model_name = 'GQCNN-4.0-PJ-'+str(iteration)+'-'+method

    save_datetime = False
    backend = "tf"
    __file__ = os.path.abspath('')

    if not os.path.isabs(config_filename):
        config_filename = os.path.join(os.getcwd(), config_filename)

    model_dir = os.path.join(model_dir, base_model_name)

    # Open train config.
    train_config = YamlConfig(config_filename)
    if seed is not None:
        train_config["seed"] = seed
        train_config["gqcnn"]["seed"] = seed
    if tensorboard_port is not None:
        train_config["tensorboard_port"] = tensorboard_port
    gqcnn_params = train_config["gqcnn"]

    # Set visible devices.
    if "gpu_list" in train_config:
        gqcnn_utils.set_cuda_visible_devices(train_config["gpu_list"])

    # Train the network.
    gqcnn = get_gqcnn_model(backend)(gqcnn_params)
    return gqcnn.load(model_dir)

def train_policy(iteration,method=""):
    dataset_dir = "dynamic_grasp_data/"+method+"/"
    split_name = "image_wise"
    output_dir = "grasp_learner/models/"
    if iteration == 0:
        model_dir = "grasp_experts/models/"
    else:
        model_dir = "grasp_learner/models/"
    tensorboard_port = None
    seed = 1
    config_filename = "grasp_experts/cfg/finetune_dex-net_4.0_pj.yaml"
    name = "GQCNN-4.0-PJ-"+str(iteration+1)+'-'+method

    if iteration == 0:
        base_model_name = "GQCNN-4.0-PJ"
    else:
        base_model_name = 'GQCNN-4.0-PJ-'+str(iteration)+'-'+method

    save_datetime = False
    backend = "tf"
    __file__ = os.path.abspath('')

    # Set default output dir.
    if output_dir is None:
        output_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)),
                                  "../models")

    # Set default config filename.
    if config_filename is None:
        config_filename = os.path.join(
            os.path.dirname(os.path.realpath(__file__)), "..",
            "cfg/finetune.yaml")

    # Set default model dir.
    if model_dir is None:
        model_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)),
                                 "../models")

    # Turn relative paths absolute.
    if not os.path.isabs(dataset_dir):
        dataset_dir = os.path.join(os.getcwd(), dataset_dir)
    if not os.path.isabs(output_dir):
        output_dir = os.path.join(os.getcwd(), output_dir)
    if not os.path.isabs(config_filename):
        config_filename = os.path.join(os.getcwd(), config_filename)

    # Create full path to the pre-trained model.
    model_dir = os.path.join(model_dir, base_model_name)

    # Create output dir if necessary.
    utils.mkdir_safe(output_dir)

    # Open train config.
    train_config = YamlConfig(config_filename)
    if seed is not None:
        train_config["seed"] = seed
        train_config["gqcnn"]["seed"] = seed
    if tensorboard_port is not None:
        train_config["tensorboard_port"] = tensorboard_port
    gqcnn_params = train_config["gqcnn"]

    # Create a unique output folder based on the date and time.
    if save_datetime:
        # Create output dir.
        unique_name = time.strftime("%Y%m%d-%H%M%S")
        output_dir = os.path.join(output_dir, unique_name)
        utils.mkdir_safe(output_dir)

    # Set visible devices.
    if "gpu_list" in train_config:
        gqcnn_utils.set_cuda_visible_devices(train_config["gpu_list"])

    # Train the network.
    start_time = time.time()
    gqcnn = get_gqcnn_model(backend)(gqcnn_params)
    trainer = get_gqcnn_trainer(backend)(gqcnn,
                                         dataset_dir,
                                         split_name,
                                         output_dir,
                                         train_config,
                                         name=name)
    trainer.finetune(model_dir)
    shutil.rmtree(os.path.join(dataset_dir,"splits"), ignore_errors=True)

In [3]:
rospy.init_node("ik_pick_and_place_demo")

In [4]:
limb = "left"
hover_distance = 0.15
pnp = PickAndPlace(limb, hover_distance)
pnp.move_to_start()

[INFO] [1567676835.518830, 16.123000]: Robot Enabled


In [5]:
rospy.wait_for_service('/gazebo/spawn_sdf_model')
rospy.wait_for_service('/gazebo/spawn_urdf_model')
spawn_sdf = rospy.ServiceProxy('/gazebo/spawn_sdf_model', gazebo_msgs.srv.SpawnModel)
spawn_urdf = rospy.ServiceProxy('/gazebo/spawn_urdf_model', gazebo_msgs.srv.SpawnModel)
delete_model = rospy.ServiceProxy('/gazebo/delete_model', gazebo_msgs.srv.DeleteModel)
set_model = rospy.ServiceProxy('/gazebo/set_model_state', gazebo_msgs.srv.SetModelState)

camera_quaternion = tf.transformations.quaternion_from_euler(0., np.pi / 2., 0.)
spawn_gazebo_model(spawn_urdf, "realsense", "rs200", 0.65, 0.3, 0.7825+0.7, 
                   qx=camera_quaternion[0],qy=camera_quaternion[1],qz=camera_quaternion[2],qw=camera_quaternion[3])
spawn_gazebo_model(spawn_sdf, "cafe_table", "cafe_table", 0.8, 0.3, 0.0)

model_name_list = ["CokePlasticLarge_800_tex","YellowSaltCylinderSmall_800_tex","Marjoram_800_tex","BathDetergent_800_tex"]
model_name = "obj_10"
number_of_object_per_model=1
number_of_object=0

for i in range(number_of_object_per_model):
    rnd_x = 0.025*(2.*np.random.uniform()-1.) + 0.6
    rnd_y = 0.025*(2.*np.random.uniform()-1.) + 0.3
    rnd_z = 0.9
    rnd_roll = 2.*np.pi*np.random.uniform()
    rnd_pitch = 2.*np.pi*np.random.uniform()
    rnd_ywa = 2.*np.pi*np.random.uniform()
    rnd_quaternion = tf.transformations.quaternion_from_euler(rnd_roll,rnd_pitch,rnd_ywa)
    spawn_gazebo_model(spawn_sdf, model_name, "object"+str(i), 
                       rnd_x, rnd_y, rnd_z,
                       rnd_quaternion[0],rnd_quaternion[1],rnd_quaternion[2],rnd_quaternion[3])
    rospy.sleep(0.01)
number_of_object+=number_of_object_per_model 

In [6]:
depth_cam_rotation_matrix = np.zeros([4,4])
depth_cam_rotation_matrix[1,0] = -1.
depth_cam_rotation_matrix[0,1] = -1.
depth_cam_rotation_matrix[2,2] = -1.
depth_cam_rotation_matrix[3,3] = 1.
depth_cam_rotation_matrix[0,3] = .65
depth_cam_rotation_matrix[1,3] = 0.3-.03
depth_cam_rotation_matrix[2,3] = 1.4825

In [ ]:
for method in ['Greedy','EXP3']:
    total_data_cnt = 0
    test_size = 5
    batch_size = 20
    num_trials = 5
    num_seeds = 5
    exp_performance_list = np.zeros([num_seeds, num_trials])
    best_performance_list = np.zeros([num_seeds, num_trials])

    base_gqcnn = load_gqcnn_model(0,method)
    base_gqcnn.open_session()

    for j in range(172):
        grasp_metrics = np.load('./dynamic_grasp_data/training/tensors/grasp_metrics_%05d.npz'%j)['arr_0']
        tf_depth_ims = np.load('./dynamic_grasp_data/training/tensors/tf_depth_ims_%05d.npz'%j)['arr_0']
        grasps = np.load('./dynamic_grasp_data/training/tensors/grasps_%05d.npz'%j)['arr_0']

        current_data_cnt = len(grasp_metrics)
        rand_idx = np.random.permutation(current_data_cnt)

        grasp_metrics = grasp_metrics[rand_idx]
        tf_depth_ims = tf_depth_ims[rand_idx]
        grasps = grasps[rand_idx]
        tf_features = base_gqcnn.featurize(tf_depth_ims,feature_layer='fc3')
        sig_features = np.minimum(np.min(np.std(tf_features)),0.6)
        tf_features += 4*sig_features*np.random.randn(tf_features.shape[0],tf_features.shape[1])
        dpp = DPP(tf_features)
        dpp.compute_kernel(kernel_type='rbf', sigma=sig_features)
        sampled_idx = dpp.sample_k(batch_size)

        with open(os.path.join("./dynamic_grasp_data/"+method+"-GQ/tensors", 'grasp_metrics_%05d.npz'%total_data_cnt), 'wb') as f_dim:
            np.savez(f_dim, grasp_metrics[sampled_idx])

        with open(os.path.join("./dynamic_grasp_data/"+method+"-GQ/tensors", 'tf_depth_ims_%05d.npz'%total_data_cnt), 'wb') as f_grasp:
            np.savez(f_grasp, tf_depth_ims[sampled_idx])

        with open(os.path.join("./dynamic_grasp_data/"+method+"-GQ/tensors", 'grasps_%05d.npz'%total_data_cnt), 'wb') as f_metric:
            np.savez(f_metric, grasps[sampled_idx])

        total_data_cnt+=1
    base_gqcnn.close_session()
    print(total_data_cnt)
    
    for k in range(number_of_object):
        set_object_state(set_model,"object"+str(k),0.8)
    rospy.sleep(2.)
    pnp._gripper.set_holding_force(15.)
    pnp._gripper.set_moving_force(15.)
    for seed in range(num_seeds):
        np.random.seed(seed)
        for j in range(num_trials):
            rospy.sleep(1.)
            cropped_depth_imgs = np.zeros([batch_size,crop_size*2,crop_size*2,1])
            hand_poses = np.zeros([batch_size,6])
            grasp_results = np.zeros([batch_size,])
            test_grasp_results = np.zeros([test_size,])
            policy, best_action, sample_action = load_policy(j, wx=wx, wy=wy, method=method)
            for i in range(test_size):
                pnp.move_to_start(open_gripper=True)
                observation = get_rgbd_image()
                color_data = observation["color"][cy-wy:cy+wy,cx-wx:cx+wx,:]
                depth_data = np.float32(observation["depth"][cy-wy:cy+wy,cx-wx:cx+wx])*0.001

                try:
                    action = best_action(policy, depth_data, color_data)
                except NoValidGraspsException:
                    for k in range(number_of_object):
                        set_object_state(set_model,"object"+str(k))
                    rospy.sleep(2.)
                    continue

                center_px, center_py = action.grasp.center.data
                center_py = int(center_py)
                center_px = int(center_px)
                center_depth = depth_data[center_py,center_px]
                gripper_angle = action.grasp.angle

                center_px = center_px - wx + cx
                center_py = center_py - wy + cy

                depth_camera_info = observation["depth_camera_info"]
                K = np.asarray(depth_camera_info.K).reshape([3,3])

                projected_eef_position = np.asarray([center_px,center_py,1.])
                scaled_eef_position = np.matmul(np.linalg.inv(K),projected_eef_position)
                scaled_eef_position = scaled_eef_position*center_depth
                scaled_eef_position[2] = action.grasp.depth
                eef_position = np.matmul(depth_cam_rotation_matrix[:3,:3],scaled_eef_position) + depth_cam_rotation_matrix[:3,3]

                approaching_axis = np.asarray([0.,0.,-1.])
                gripper_axis = np.asarray([-np.sin(gripper_angle),-np.cos(gripper_angle),0.])

                x_axis = np.cross(gripper_axis, approaching_axis) 
                y_axis = gripper_axis
                z_axis = approaching_axis

                gripper_rotation_mtx = np.identity(4)
                gripper_rotation_mtx[:3,0] = x_axis
                gripper_rotation_mtx[:3,1] = y_axis
                gripper_rotation_mtx[:3,2] = z_axis
                eef_quaternion = tf.transformations.quaternion_from_matrix(gripper_rotation_mtx)

                grasp_pose = geometry_msgs.msg.Pose()
                grasp_pose.position.x = eef_position[0]
                grasp_pose.position.y = eef_position[1]
                grasp_pose.position.z = eef_position[2] - 0.9115
                grasp_pose.orientation.x = eef_quaternion[0]
                grasp_pose.orientation.y = eef_quaternion[1]
                grasp_pose.orientation.z = eef_quaternion[2]
                grasp_pose.orientation.w = eef_quaternion[3]

                if eef_position[2] - 0.02 > 0.755:
                    grasp_pose.position.z = 0.757 - 0.9115

                pnp.move_to_ready(open_gripper=True)
                pnp.pick(grasp_pose)
                pnp.move_to_ready(open_gripper=False)
                pnp.move_to_start(open_gripper=False)

                for k in range(number_of_object):
                    grasp_result = get_grasp_result("object"+str(k))
                    if grasp_result:
                        break
                cropped_depth_imgs[i,:,:,:] = np.float32(observation["depth"][center_py-crop_size:center_py+crop_size,center_px-crop_size:center_px+crop_size,np.newaxis])*0.001        
                hand_poses[i,2] = action.grasp.depth
                hand_poses[i,3] = gripper_angle
                test_grasp_results[i] = grasp_result

                print("===================================================")
                print("[%d][%d] Total : %d/%d, Current : %d"%(seed,j,np.sum(test_grasp_results),i+1,grasp_result))
                print("===================================================")

                if (i+1)%number_of_object == 0:
                    for k in range(number_of_object):
                        set_object_state(set_model,"object"+str(k))
                    rospy.sleep(2.)

            best_performance_list[seed,j] = np.sum(test_grasp_results)/len(test_grasp_results)
            print("Best Result: %f"%best_performance_list[seed,j])

            for i in range(batch_size):
                pnp.move_to_start(open_gripper=True)
                observation = get_rgbd_image()
                color_data = observation["color"][cy-wy:cy+wy,cx-wx:cx+wx,:]
                depth_data = np.float32(observation["depth"][cy-wy:cy+wy,cx-wx:cx+wx])*0.001

                try:
                    action = sample_action(policy, depth_data, color_data)
                except NoValidGraspsException:
                    for k in range(number_of_object):
                        set_object_state(set_model,"object"+str(k),0.8)
                    rospy.sleep(2.)
                    continue

                center_px, center_py = action.grasp.center.data
                center_py = int(center_py)
                center_px = int(center_px)
                center_depth = depth_data[center_py,center_px]
                gripper_angle = action.grasp.angle

                center_px = center_px - wx + cx
                center_py = center_py - wy + cy

                depth_camera_info = observation["depth_camera_info"]
                K = np.asarray(depth_camera_info.K).reshape([3,3])

                projected_eef_position = np.asarray([center_px,center_py,1.])
                scaled_eef_position = np.matmul(np.linalg.inv(K),projected_eef_position)
                scaled_eef_position = scaled_eef_position*center_depth
                scaled_eef_position[2] = action.grasp.depth
                eef_position = np.matmul(depth_cam_rotation_matrix[:3,:3],scaled_eef_position) + depth_cam_rotation_matrix[:3,3]

                approaching_axis = np.asarray([0.,0.,-1.])
                gripper_axis = np.asarray([-np.sin(gripper_angle),-np.cos(gripper_angle),0.])

                x_axis = np.cross(gripper_axis, approaching_axis) 
                y_axis = gripper_axis
                z_axis = approaching_axis

                gripper_rotation_mtx = np.identity(4)
                gripper_rotation_mtx[:3,0] = x_axis
                gripper_rotation_mtx[:3,1] = y_axis
                gripper_rotation_mtx[:3,2] = z_axis
                eef_quaternion = tf.transformations.quaternion_from_matrix(gripper_rotation_mtx)

                grasp_pose = geometry_msgs.msg.Pose()
                grasp_pose.position.x = eef_position[0]
                grasp_pose.position.y = eef_position[1]
                grasp_pose.position.z = eef_position[2] - 0.9115
                grasp_pose.orientation.x = eef_quaternion[0]
                grasp_pose.orientation.y = eef_quaternion[1]
                grasp_pose.orientation.z = eef_quaternion[2]
                grasp_pose.orientation.w = eef_quaternion[3]

                if eef_position[2] - 0.02 > 0.755:
                    grasp_pose.position.z = 0.757 - 0.9115

                pnp.move_to_ready(open_gripper=True)
                pnp.pick(grasp_pose)
                pnp.move_to_ready(open_gripper=False)
                pnp.move_to_start(open_gripper=False)

                for k in range(number_of_object):
                    grasp_result = get_grasp_result("object"+str(k))
                    if grasp_result:
                        break

                cropped_depth_imgs[i,:,:,:] = np.float32(observation["depth"][center_py-crop_size:center_py+crop_size,center_px-crop_size:center_px+crop_size,np.newaxis])*0.001        
                hand_poses[i,2] = action.grasp.depth
                hand_poses[i,3] = gripper_angle
                grasp_results[i] = grasp_result

                print("===================================================")
                print("[%d][%d] Total : %d/%d, Current : %d"%(seed,j,np.sum(grasp_results),i+1,grasp_result))
                print("===================================================")

                if (i+1)%number_of_object == 0:
                    for k in range(number_of_object):    
                        set_object_state(set_model,"object"+str(k),0.9)
                    rospy.sleep(2.)

            with open(os.path.join(target_dir+method+"-GQ/tensors", fname[0]+'%05d.npz'%(total_data_cnt+j)), 'wb') as f_dim:
                np.savez(f_dim, cropped_depth_imgs)
            with open(os.path.join(target_dir+method+"-GQ/tensors", fname[1]+'%05d.npz'%(total_data_cnt+j)), 'wb') as f_grasp:
                np.savez(f_grasp, hand_poses)
            with open(os.path.join(target_dir+method+"-GQ/tensors", fname[2]+'%05d.npz'%(total_data_cnt+j)), 'wb') as f_metric:
                np.savez(f_metric, grasp_results)

            train_policy(j, method=method+"-GQ")
            exp_performance_list[seed,j] = np.sum(grasp_results)/len(grasp_results)
            print("Best Result: %f"%exp_performance_list[seed,j])

        for j in range(num_trials):
            os.remove(os.path.join(target_dir+method+"-GQ/tensors", fname[0]+'%05d.npz'%(total_data_cnt+j)))
            os.remove(os.path.join(target_dir+method+"-GQ/tensors", fname[1]+'%05d.npz'%(total_data_cnt+j)))
            os.remove(os.path.join(target_dir+method+"-GQ/tensors", fname[2]+'%05d.npz'%(total_data_cnt+j)))

    with open("./"+method+"-GQ-"+model_name+"-results.npz", 'wb') as f_res:
        np.savez(f_res, exp_performance_list, best_performance_list)

    for j in range(total_data_cnt):
        os.remove(os.path.join(target_dir+method+"-GQ/tensors", fname[0]+'%05d.npz'%(j)))
        os.remove(os.path.join(target_dir+method+"-GQ/tensors", fname[1]+'%05d.npz'%(j)))
        os.remove(os.path.join(target_dir+method+"-GQ/tensors", fname[2]+'%05d.npz'%(j)))

172
/home/dof6/catkin_ws/src/grasp_adaptation/scripts/grasp_experts/models/GQCNN-4.0-PJ
OrderedDict([('num_seed_samples', 128), ('num_gmm_samples', 64), ('num_iters', 3), ('gmm_refit_p', 0.25), ('gmm_component_frac', 0.4), ('gmm_reg_covar', 0.01), ('deterministic', 1), ('gripper_width', 0.05), ('sampling', OrderedDict([('type', 'antipodal_depth'), ('friction_coef', 1.0), ('depth_grad_thresh', 0.0025), ('depth_grad_gaussian_sigma', 1.0), ('downsample_rate', 4), ('max_rejection_samples', 4000), ('max_dist_from_center', 160), ('min_dist_from_boundary', 68), ('min_grasp_dist', 2.5), ('angle_dist_weight', 5.0), ('depth_sampling_mode', 'uniform'), ('depth_samples_per_grasp', 3), ('depth_sample_win_height', 1), ('depth_sample_win_width', 1), ('min_depth_offset', 0.015), ('max_depth_offset', 0.05), ('gripper_width', 0.05)])), ('metric', OrderedDict([('type', 'gqcnn'), ('gqcnn_model', '/home/dof6/catkin_ws/src/grasp_adaptation/scripts/grasp_experts/models/GQCNN-4.0-PJ'), ('crop_height', 96), ('

Grasp Quality : 0.506170
[INFO] [1567677500.205165, 651.377000]: 457: No Contact
[0][1] Total : 0/1, Current : 0
Grasp Quality : 0.537473
[INFO] [1567677523.275739, 674.377000]: 472: Contact with Gripper
[0][1] Total : 1/2, Current : 1
Grasp Quality : 0.511454
[INFO] [1567677546.897998, 697.951000]: 493: Contact with Gripper
[0][1] Total : 2/3, Current : 1
Grasp Quality : 0.483502
[INFO] [1567677571.871159, 722.892000]: 510: No Contact with Gripper
[0][1] Total : 2/4, Current : 0
Grasp Quality : 0.553425
[INFO] [1567677595.010582, 745.979000]: 535: No Contact
[0][1] Total : 2/5, Current : 0
Best Result: 0.400000
Grasp Quality : 0.422089
[INFO] [1567677620.641073, 770.389000]: 557: No Contact with Gripper
[0][1] Total : 0/1, Current : 0
Grasp Quality : 0.434345
[INFO] [1567677644.600012, 794.246000]: 575: No Contact
[0][1] Total : 0/2, Current : 0
Grasp Quality : 0.521224
[INFO] [1567677667.265868, 816.862000]: 588: Contact with Gripper
[0][1] Total : 1/3, Current : 1
Grasp Quality : 0.

Grasp Quality : 0.578031
[INFO] [1567678268.742438, 1393.191000]: 1089: No Contact with Gripper
[0][2] Total : 1/2, Current : 0
Grasp Quality : 0.564554
[INFO] [1567678291.218702, 1415.646000]: 1113: No Contact with Gripper
[0][2] Total : 1/3, Current : 0
Grasp Quality : 0.570533
[INFO] [1567678315.316826, 1439.718000]: 1138: Contact with Gripper
[0][2] Total : 2/4, Current : 1
Grasp Quality : 0.503761
[INFO] [1567678337.571130, 1461.875000]: 1152: Contact with Gripper
[0][2] Total : 3/5, Current : 1
Grasp Quality : 0.601060
[INFO] [1567678361.679083, 1485.937000]: 1177: Contact with Gripper
[0][2] Total : 4/6, Current : 1
Grasp Quality : 0.482313
[INFO] [1567678385.925873, 1510.154000]: 1200: No Contact
[0][2] Total : 4/7, Current : 0
Grasp Quality : 0.605842
[INFO] [1567678411.853933, 1535.019000]: 1222: Contact with Gripper
[0][2] Total : 5/8, Current : 1
Grasp Quality : 0.586926
[INFO] [1567678434.717360, 1557.798000]: 1243: Contact with Gripper
[0][2] Total : 6/9, Current : 1
Gras

Grasp Quality : 0.619708
[INFO] [1567679021.055136, 2131.543000]: 1693: No Contact
[0][3] Total : 5/8, Current : 0
Grasp Quality : 0.652130
[INFO] [1567679045.512464, 2155.946000]: 1714: No Contact with Gripper
[0][3] Total : 5/9, Current : 0
Grasp Quality : 0.635982
[INFO] [1567679069.606489, 2179.969000]: 1726: No Contact
[0][3] Total : 5/10, Current : 0
Grasp Quality : 0.619206
[INFO] [1567679114.947943, 2202.482000]: 1740: No Contact with Gripper
[0][3] Total : 5/11, Current : 0
Grasp Quality : 0.648977
[INFO] [1567679139.289158, 2226.778000]: 1757: No Contact
[0][3] Total : 5/12, Current : 0
Grasp Quality : 0.581521
[INFO] [1567679163.435409, 2250.883000]: 1774: Contact with Gripper
[0][3] Total : 6/13, Current : 1
Grasp Quality : 0.626196
[INFO] [1567679186.868940, 2274.279000]: 1791: No Contact with Gripper
[0][3] Total : 6/14, Current : 0
Grasp Quality : 0.594841
[INFO] [1567679209.507963, 2296.897000]: 1810: Contact with Gripper
[0][3] Total : 7/15, Current : 1
Grasp Quality :

Grasp Quality : 0.687995
[INFO] [1567679791.513702, 2859.058000]: 2296: No Contact
[0][4] Total : 4/14, Current : 0
Grasp Quality : 0.692841
[INFO] [1567679815.930039, 2883.406000]: 2313: No Contact with Gripper
[0][4] Total : 4/15, Current : 0
Grasp Quality : 0.679307
[INFO] [1567679841.621287, 2907.270000]: 2337: No Contact with Gripper
[0][4] Total : 4/16, Current : 0
Grasp Quality : 0.648389
[INFO] [1567679864.307472, 2929.931000]: 2359: Contact with Gripper
[0][4] Total : 5/17, Current : 1
Grasp Quality : 0.627875
[INFO] [1567679887.030951, 2952.609000]: 2376: No Contact with Gripper
[0][4] Total : 5/18, Current : 0
Grasp Quality : 0.643759
[INFO] [1567679910.870438, 2976.260000]: 2402: Contact with Gripper
[0][4] Total : 6/19, Current : 1
Grasp Quality : 0.550605
[INFO] [1567679934.902362, 3000.244000]: 2428: No Contact with Gripper
[0][4] Total : 6/20, Current : 0
Best Result: 0.300000
/home/dof6/catkin_ws/src/grasp_adaptation/scripts/grasp_experts/models/GQCNN-4.0-PJ
OrderedDic

...

Grasp Quality : 0.661968
[INFO] [1567690655.550362, 13415.382000]: 12077: No Contact with Gripper
[4][2] Total : 4/13, Current : 0
Grasp Quality : 0.494126
[INFO] [1567690680.016953, 13439.773000]: 12095: No Contact
[4][2] Total : 4/14, Current : 0
Grasp Quality : 0.572019
[INFO] [1567690702.492696, 13462.199000]: 12111: No Contact with Gripper
[4][2] Total : 4/15, Current : 0
Grasp Quality : 0.647908
[INFO] [1567690726.773975, 13486.445000]: 12129: Contact with Gripper
[4][2] Total : 5/16, Current : 1
Grasp Quality : 0.652468
[INFO] [1567690749.370056, 13508.977000]: 12150: No Contact
[4][2] Total : 5/17, Current : 0
Grasp Quality : 0.575601
[INFO] [1567690772.200298, 13531.770000]: 12173: Contact with Gripper
[4][2] Total : 6/18, Current : 1
Grasp Quality : 0.639949
[INFO] [1567690796.302882, 13555.845000]: 12187: Contact with Gripper
[4][2] Total : 7/19, Current : 1
Grasp Quality : 0.571045
[INFO] [1567690821.040714, 13580.521000]: 12215: Contact with Gripper
[4][2] Total : 8/20, Cu

Grasp Quality : 0.642465
[INFO] [1567691370.526884, 14121.986000]: 12665: Contact with Gripper
[4][3] Total : 9/18, Current : 1
Grasp Quality : 0.552796
[INFO] [1567691395.091815, 14146.527000]: 12691: No Contact
[4][3] Total : 9/19, Current : 0
Grasp Quality : 0.530023
[INFO] [1567691420.681812, 14171.467000]: 12707: Contact with Gripper
[4][3] Total : 10/20, Current : 1
Best Result: 0.500000
/home/dof6/catkin_ws/src/grasp_adaptation/scripts/grasp_learner/models/GQCNN-4.0-PJ-4-Greedy-GQ
OrderedDict([('num_seed_samples', 128), ('num_gmm_samples', 64), ('num_iters', 3), ('gmm_refit_p', 0.25), ('gmm_component_frac', 0.4), ('gmm_reg_covar', 0.01), ('deterministic', 1), ('gripper_width', 0.05), ('sampling', OrderedDict([('type', 'antipodal_depth'), ('friction_coef', 1.0), ('depth_grad_thresh', 0.0025), ('depth_grad_gaussian_sigma', 1.0), ('downsample_rate', 4), ('max_rejection_samples', 4000), ('max_dist_from_center', 160), ('min_dist_from_boundary', 68), ('min_grasp_dist', 2.5), ('angle_d

Grasp Quality : 0.872683
[INFO] [1567692072.244378, 14811.647000]: 13232: No Contact with Gripper
[0][0] Total : 0/1, Current : 0
Grasp Quality : 0.618734
[INFO] [1567692094.802083, 14834.175000]: 13254: Contact with Gripper
[0][0] Total : 1/2, Current : 1
Grasp Quality : 0.562514
[INFO] [1567692116.959336, 14856.297000]: 13265: No Contact
[0][0] Total : 1/3, Current : 0
Grasp Quality : 0.765374
[INFO] [1567692140.899092, 14880.144000]: 13285: No Contact with Gripper
[0][0] Total : 1/4, Current : 0
Grasp Quality : 0.742020
[INFO] [1567692165.645413, 14904.747000]: 13301: Contact with Gripper
[0][0] Total : 2/5, Current : 1
Best Result: 0.400000
Grasp Quality : 0.002621
[INFO] [1567692189.266684, 14928.340000]: 13317: No Contact
[0][0] Total : 0/1, Current : 0
Grasp Quality : 0.912376
[INFO] [1567692211.950479, 14950.982000]: 13336: Contact with Gripper
[0][0] Total : 1/2, Current : 1
Grasp Quality : 0.668397
[INFO] [1567692235.707534, 14974.714000]: 13361: No Contact with Gripper
[0][0

Grasp Quality : 0.539787
[INFO] [1567692788.920269, 15519.943000]: 13809: Contact with Gripper
[0][1] Total : 1/2, Current : 1
Grasp Quality : 0.590583
[INFO] [1567692811.315651, 15542.255000]: 13825: No Contact
[0][1] Total : 1/3, Current : 0
Grasp Quality : 0.570836
[INFO] [1567692833.709716, 15564.620000]: 13842: No Contact with Gripper
[0][1] Total : 1/4, Current : 0
Grasp Quality : 0.581004
[INFO] [1567692856.475688, 15587.347000]: 13866: No Contact
[0][1] Total : 1/5, Current : 0
Grasp Quality : 0.623719
[INFO] [1567692888.636726, 15612.280000]: 13893: No Contact
[0][1] Total : 1/6, Current : 0
Grasp Quality : 0.603426
[INFO] [1567692911.198362, 15634.816000]: 13922: Contact with Gripper
[0][1] Total : 2/7, Current : 1
Grasp Quality : 0.459169
[INFO] [1567692935.263458, 15658.058000]: 13940: Contact with Gripper
[0][1] Total : 3/8, Current : 1
Grasp Quality : 0.578775
[INFO] [1567692960.063762, 15682.820000]: 13971: No Contact with Gripper
[0][1] Total : 3/10, Current : 0
Grasp Q

Grasp Quality : 0.536039
[INFO] [1567693546.237714, 16256.295000]: 14489: Contact with Gripper
[0][2] Total : 6/10, Current : 1
Grasp Quality : 0.637880
[INFO] [1567693570.755711, 16280.789000]: 14503: No Contact with Gripper
[0][2] Total : 6/11, Current : 0
Grasp Quality : 0.534908
[INFO] [1567693593.333313, 16303.315000]: 14523: Contact with Gripper
[0][2] Total : 7/12, Current : 1
Grasp Quality : 0.640756
[INFO] [1567693615.956032, 16325.896000]: 14550: No Contact with Gripper
[0][2] Total : 7/13, Current : 0
Grasp Quality : 0.546843
[INFO] [1567693640.570729, 16350.479000]: 14578: Contact with Gripper
[0][2] Total : 8/14, Current : 1
Grasp Quality : 0.477056
[INFO] [1567693665.010617, 16374.894000]: 14602: Contact with Gripper
[0][2] Total : 9/15, Current : 1
Grasp Quality : 0.642249
[INFO] [1567693689.927960, 16399.774000]: 14626: No Contact
[0][2] Total : 9/16, Current : 0
Grasp Quality : 0.506205
[INFO] [1567693714.574654, 16424.376000]: 14652: Contact with Gripper
[0][2] Total 

In [ ]:
print(exp_performance_list[:5,:])
print(np.mean(exp_performance_list[:5,:],axis=0))
print(np.std(exp_performance_list[:5,:],axis=0))
plt.plot(np.mean(exp_performance_list,axis=0))

In [ ]:
print(best_performance_list)
print(np.mean(best_performance_list,axis=0))
print(np.std(best_performance_list,axis=0))
plt.plot(np.mean(best_performance_list,axis=0))